In [1]:
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder

## Conditions
### spinal_canal_stenosis
### neural_foraminal_narrowing
#### left/right
### subarticular_stenosis
#### left/right

## Levels of vertebrae
#### l1_l2
#### l2_l3
#### l3_l4
#### l4_l5
#### l5_s1

> Each Study has series of images for these three conditions. The series description tells us what type of image is in the dcm file

### Reading the available training metadata files using polars
### Bringing the metadata files in a desired shape

In [4]:
train_meta_data = pl.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv'\
                              , low_memory=True)
train_series_desc = pl.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv'\
                                , low_memory=True)
train_label_coordinates = pl.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv'\
                                      , low_memory=True)

train_meta_data_w_srs_desc = dd.sql("select b.series_description, b.series_id, a.* from train_meta_data a \
left join train_series_desc b on a.study_id = b.study_id").pl()

melt_cols = train_meta_data_w_srs_desc.columns[3:]
train_meta_data_melted = train_meta_data_w_srs_desc.melt(id_vars=["study_id","series_id","series_description"], value_vars=melt_cols)
train_meta_data_melted.columns = ['study_id', 'series_id', 'series_description', 'condition_vert_level', 'severity']

train_label_coordinates_formatted = dd.sql("select study_id, series_id, instance_number \
,lower(replace(condition,' ','_')||'_'||replace(level,'/','_')) as condition, x, y \
from train_label_coordinates").pl()

full_training_data = dd.sql("select t1.study_id, t1.series_id, t1.series_description, t2.instance_number \
, t1.condition_vert_level, t1.severity, t2.x, t2.y \
from train_meta_data_melted t1 \
left join train_label_coordinates_formatted t2 on t1.study_id = t2.study_id \
and t1.condition_vert_level = t2.condition \
").pl()

print(full_training_data.shape)

(157350, 8)


In [33]:
dd.sql("select * from train_meta_data_melted where study_id = 3008676218 ").pl()

study_id,series_id,series_description,condition_vert_level,severity
i64,i64,str,str,str
3008676218,3636216534,"""Axial T2""","""spinal_canal_stenosis_l1_l2""",null
3008676218,542282425,"""Sagittal T1""","""spinal_canal_stenosis_l1_l2""",null
3008676218,3636216534,"""Axial T2""","""spinal_canal_stenosis_l2_l3""",null
3008676218,542282425,"""Sagittal T1""","""spinal_canal_stenosis_l2_l3""",null
3008676218,3636216534,"""Axial T2""","""spinal_canal_stenosis_l3_l4""",null
3008676218,542282425,"""Sagittal T1""","""spinal_canal_stenosis_l3_l4""",null
3008676218,3636216534,"""Axial T2""","""spinal_canal_stenosis_l4_l5""",null
3008676218,542282425,"""Sagittal T1""","""spinal_canal_stenosis_l4_l5""",null
3008676218,3636216534,"""Axial T2""","""spinal_canal_stenosis_l5_s1""",null


#### Checking a specific study/patient's scans and the corresponding details, only to get a better sense of the data

In [29]:
pl.Config(fmt_str_lengths=100)
pl.Config.set_tbl_rows(1000)

#print(train_meta_data_melted.shape)
# print(train_meta_data.filter(pl.col('study_id')==100206310).transpose(include_header=True))
#print(train_meta_data_melted.filter(pl.col('study_id')==100206310))
# print(train_series_desc.filter(pl.col('study_id')==100206310))
#print(train_label_coordinates_formatted.shape)
#print(train_label_coordinates_formatted.filter(pl.col('study_id')==100206310))
#print(train_series_coordinates.filter(pl.col('study_id')==100206310).sort(['study_id','series_id']))
print(full_training_data.filter(pl.col('study_id')==100206310).sort(['study_id','series_id']))

shape: (75, 8)
┌───────────┬────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┐
│ study_id  ┆ series_id  ┆ series_des ┆ instance_n ┆ condition ┆ severity  ┆ x         ┆ y         │
│ ---       ┆ ---        ┆ cription   ┆ umber      ┆ _vert_lev ┆ ---       ┆ ---       ┆ ---       │
│ i64       ┆ i64        ┆ ---        ┆ ---        ┆ el        ┆ str       ┆ f64       ┆ f64       │
│           ┆            ┆ str        ┆ i64        ┆ ---       ┆           ┆           ┆           │
│           ┆            ┆            ┆            ┆ str       ┆           ┆           ┆           │
╞═══════════╪════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 100206310 ┆ 1012284084 ┆ Axial T2   ┆ 13         ┆ left_neur ┆ Normal/Mi ┆ 270.34225 ┆ 148.22145 │
│           ┆            ┆            ┆            ┆ al_forami ┆ ld        ┆           ┆ 9         │
│           ┆            ┆            ┆            ┆ nal_narro ┆           ┆

In [5]:
condition_list = dd.sql("select distinct condition_vert_level from full_training_data").pl().to_series().to_list()

In [11]:
dd.sql("select series_description, severity, count(distinct(study_id||'/'||series_id)) as ss_count from full_training_data \
where condition_vert_level = 'left_neural_foraminal_narrowing_l1_l2' and severity is not null and severity != 'Severe' group by series_description, severity").pl()

series_description,severity,ss_count
str,str,i64
"""Axial T2""","""Normal/Mild""",2256
"""Sagittal T2/STIR""","""Normal/Mild""",1907
"""Sagittal T1""","""Normal/Mild""",1915
"""Sagittal T1""","""Moderate""",63
"""Sagittal T2/STIR""","""Moderate""",63
"""Axial T2""","""Moderate""",80


In [10]:
dd.sql("select series_description, count(distinct(study_id||'/'||series_id)) as ss_count from full_training_data \
where severity = 'Severe' group by series_description").pl()

series_description,ss_count
str,i64
"""Sagittal T1""",982
"""Sagittal T2/STIR""",979
"""Axial T2""",1154


In [13]:
pl.Config(fmt_str_lengths=100)
pl.Config.set_tbl_rows(1000)

dd.sql("select condition_vert_level, series_description, count(distinct(study_id||'/'||series_id)) as ss_count from full_training_data \
where severity = 'Severe' group by condition_vert_level, series_description").pl()

condition_vert_level,series_description,ss_count
str,str,i64
"""left_subarticular_stenosis_l4_l5""","""Axial T2""",538
"""left_subarticular_stenosis_l4_l5""","""Sagittal T2/STIR""",461
"""left_subarticular_stenosis_l4_l5""","""Sagittal T1""",462
"""left_subarticular_stenosis_l5_s1""","""Sagittal T2/STIR""",147
"""left_subarticular_stenosis_l5_s1""","""Axial T2""",171
"""left_subarticular_stenosis_l5_s1""","""Sagittal T1""",147
"""right_subarticular_stenosis_l1_l2""","""Sagittal T2/STIR""",24
"""right_subarticular_stenosis_l1_l2""","""Axial T2""",34
"""right_subarticular_stenosis_l1_l2""","""Sagittal T1""",23


In [15]:
train_files_path_root = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/'

for condition in condition_list:
    print("generating feature metadata for {0}".format(condition))

    full_feature_metadata = dd.sql("select distinct ('{0}'||study_id||'/'||series_id) as file_path_root, severity from full_training_data \
    where condition_vert_level = '{1}' and severity is not null".format(train_files_path_root, condition)).pl()

    full_feature_metadata_pd = full_feature_metadata.to_pandas()

    label_mapping = {}

    for file_path in range(full_feature_metadata_pd.shape[0]):
        study_series_path = full_feature_metadata_pd['file_path_root'][file_path]
        for image_file_path in os.listdir(study_series_path):
            full_img_path = os.path.join(study_series_path, image_file_path)
            label_mapping[full_img_path] = full_feature_metadata_pd['severity'][file_path]


    # Create a list of tuples containing the audio file paths and labels
    data = [(full_img_path, severity) for full_img_path, severity in label_mapping.items()]
    # Create a Pandas DataFrame from the list of tuples
    annotated_data = pd.DataFrame(data, columns=['full_img_path', 'severity'])

    label_encoder = LabelEncoder()

    annotated_data['encoded_severity'] = label_encoder.fit_transform(annotated_data['severity'])
    print(annotated_data.shape)
    print(annotated_data.head(5))
    
    file_name = "{0}_feature_metadata".format(condition)

    annotated_data.to_csv("{0}.csv".format(file_name), index=False)
    print("saved file {0}".format(file_name))

generating feature metadata for left_subarticular_stenosis_l2_l3
(143288, 3)
                                       full_img_path     severity  \
0  /kaggle/input/rsna-2024-lumbar-spine-degenerat...  Normal/Mild   
1  /kaggle/input/rsna-2024-lumbar-spine-degenerat...  Normal/Mild   
2  /kaggle/input/rsna-2024-lumbar-spine-degenerat...  Normal/Mild   
3  /kaggle/input/rsna-2024-lumbar-spine-degenerat...  Normal/Mild   
4  /kaggle/input/rsna-2024-lumbar-spine-degenerat...  Normal/Mild   

   encoded_severity  
0                 1  
1                 1  
2                 1  
3                 1  
4                 1  
saved file left_subarticular_stenosis_l2_l3_feature_metadata
generating feature metadata for left_subarticular_stenosis_l3_l4
(147047, 3)
                                       full_img_path  severity  \
0  /kaggle/input/rsna-2024-lumbar-spine-degenerat...  Moderate   
1  /kaggle/input/rsna-2024-lumbar-spine-degenerat...  Moderate   
2  /kaggle/input/rsna-2024-lumbar-spine-

In [61]:
"""dicom_ds = dcmread(dcm_image_path)
IMG = dicom_ds.pixel_array"""

labels = []
features = []

for i in tqdm(range(annotated_data.shape[0]), desc = 'Outer Loop'):
    dcm_image_path = annotated_data['full_img_path'].iloc[i]
    dicom_ds = dcmread(dcm_image_path)
    features.append(dicom_ds.pixel_array)
    labels.append(annotated_data['encoded_severity'].iloc[i])

Outer Loop:   9%|▉         | 13418/147083 [05:10<51:36, 43.17it/s]  


KeyboardInterrupt: 

In [31]:
dd.sql("select condition_vert_level, series_description, severity, count(distinct(study_id)) as study_count \
, count(distinct(series_id)) as series_count from full_training_data \
group by condition_vert_level, series_description, severity order by 1,2,3").pl()

condition_vert_level,series_description,severity,study_count,series_count
str,str,str,i64,i64
"""left_neural_foraminal_narrowing_l1_l2""","""Axial T2""","""Moderate""",63,80
"""left_neural_foraminal_narrowing_l1_l2""","""Axial T2""","""Normal/Mild""",1908,2256
"""left_neural_foraminal_narrowing_l1_l2""","""Axial T2""","""Severe""",2,2
"""left_neural_foraminal_narrowing_l1_l2""","""Axial T2""",null,2,2
"""left_neural_foraminal_narrowing_l1_l2""","""Sagittal T1""","""Moderate""",63,63
"""left_neural_foraminal_narrowing_l1_l2""","""Sagittal T1""","""Normal/Mild""",1908,1915
"""left_neural_foraminal_narrowing_l1_l2""","""Sagittal T1""","""Severe""",2,2
"""left_neural_foraminal_narrowing_l1_l2""","""Sagittal T2/STIR""","""Moderate""",63,63
"""left_neural_foraminal_narrowing_l1_l2""","""Sagittal T2/STIR""","""Normal/Mild""",1907,1907


#### Looking at the number of studies and series of images available for various scan orientations, it appears that the Saggital orientations are missing for a couple of studies

In [7]:
dd.sql("select series_description, count(distinct(study_id)) as studies, count(distinct(series_id)) as series \
from train_series_desc group by series_description").pl()

series_description,studies,series
str,i64,i64
"""Axial T2""",1975,2340
"""Sagittal T1""",1973,1980
"""Sagittal T2/STIR""",1974,1974


#### Checking whether all the different conditions for all the 5 vertebrae are available in all the studies.
#### It appears that some of the condition/vertebrae combinations are not available in some of the studies
##### e.g. The images with orientation "Axial T2" for the condition "Right Subarticular Stenosis" at the vertebra "L1/L2" is available in 1812 studies out of a total of 1975 studies

In [ ]:
part_1 = list(os.listdir('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'))
# part_1 = list(filter(lambda x: x.find('.DS') == -1, part_1))
len(part_1)

In [ ]:
meta_obj = {{ 
    'study_id': x,
    'folder_path': f"/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/{x}",
    'SeriesInstanceUIDs': [] 
}
    for x in part_1 
}

In [ ]:
meta_obj

In [ ]:
"""p1 = [(x, f"/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/{x}") for x in part_1]
meta_obj = { p[0]: { 'folder_path': p[1], 
                    'SeriesInstanceUIDs': [] 
                   } 
            for p in p1 }"""

In [ ]:
for m in meta_obj:
    meta_obj[m]['SeriesInstanceUIDs'] = list(os.listdir(meta_obj[m]['folder_path']))

In [ ]:
train_series_desc.filter((pl.col('study_id') == int('100206310')) & (pl.col('series_id') == int('1792451510')))\
.select(pl.col('series_description')).item(0,0)

In [ ]:
# grabs the correspoding series descriptions
for k in tqdm(meta_obj):
    for s in meta_obj[k]['SeriesInstanceUIDs']:
        if 'SeriesDescriptions' not in meta_obj[k]:
            meta_obj[k]['SeriesDescriptions'] = []
        try:
            meta_obj[k]['SeriesDescriptions'].append(train_series_desc.filter((pl.col('study_id') == int(k)) \
                                                                              & (pl.col('series_id') == int(s)))\
                                                     .select(pl.col('series_description')).item(0,0))
        except:
            print("Failed on", s, k)

In [ ]:
# ptobj = meta_obj[list(meta_obj.keys())[10]]
ptobj = meta_obj['100206310']
ptobj

In [ ]:
ptobj['image_files'] = []

for idx, i in enumerate(ptobj['SeriesInstanceUIDs']):
    print(idx)
    print(i)
    print("**********")
    ptobj['image_files'].append(glob.glob(f"{ptobj['folder_path']}/{ptobj['SeriesInstanceUIDs'][idx]}/*.dcm"))
    
#len(images)

In [ ]:
ptobj['SeriesDescriptions'][0]

In [ ]:
ptobj['image_files'][0]

In [ ]:
ptobj

In [ ]:
for k in tqdm(meta_obj):
    meta_obj[k]['image_files'] = []
    for idx, i in enumerate(meta_obj[k]['SeriesInstanceUIDs']):
        meta_obj[k]['image_files']\
        .append(len(glob.glob(f"{meta_obj[k]['folder_path']}/{meta_obj[k]['SeriesInstanceUIDs'][idx]}/*.dcm")))

In [ ]:
meta_obj['100206310']

In [ ]:
meta_obj[list(meta_obj.keys())[1]]

In [ ]:
pl.Config(fmt_str_lengths=100)
pl.Config.set_tbl_rows(1000)
dd.sql(" \
select tsd.series_description, tlc.condition, tlc.level, \
count(distinct(tlc.study_id)) as studies, count(distinct(tlc.instance_number)) as label_defining_images, \
round(count(distinct(tlc.study_id))/1975, 5) as pct_of_total_studies, \
from train_series_desc tsd \
join train_label_coordinates tlc \
on tsd.study_id = tlc.study_id \
and tsd.series_id = tlc.series_id \
group by tsd.series_description, tlc.condition, tlc.level \
order by 1,2, 4 desc \
").pl()

#### Now let's check how many studies have moderate or severe levels of the condition "Right Subarticular Stenosis" at the vertebra "L1/L2"

In [ ]:
dd.sql("select value, count(distinct(study_id)) from train_meta_data_melted where variable = 'right_subarticular_stenosis_l1_l2' \
group by value").pl()

In [ ]:
dd.sql("select value, count(distinct(study_id)) from train_meta_data_melted where variable = 'right_subarticular_stenosis_l1_l2' \
and study_id in (select distinct(study_id) from train_label_coordinates where condition = 'Right Subarticular Stenosis'\
and level = 'L1/L2') group by value").pl()

In [ ]:
train_label_coordinates_list = dd.sql("select distinct study_id from train_meta_data_melted where variable = 'right_subarticular_stenosis_l1_l2' \
and value = 'Normal/Mild' \
and study_id in (select distinct(study_id) from train_label_coordinates where condition = 'Right Subarticular Stenosis'\
and level = 'L1/L2')").pl().to_series().unique().to_list()

train_meta_data_list = dd.sql("select distinct study_id from train_meta_data_melted where variable = 'right_subarticular_stenosis_l1_l2' and value = 'Normal/Mild' ")\
.pl().to_series().unique().to_list()

set(train_label_coordinates_list) ^ set(train_meta_data_list)

#### The above analysis shows that there are two studies related to Right Subarticular Stenosis of L1/L2 which are not present in the train_label_coordinates file

In [ ]:
dd.sql("select * from train_label_coordinates where study_id in ('3008676218','3303545110')").pl()

In [ ]:
dd.sql("select * from train_meta_data where study_id in ('3008676218','3303545110')").pl().transpose(include_header=True)

#### It seems from above, that the study 3303545110 is only present for Right Subarticular Stenosis of L5/S1 and not of L1/L2 and the study 3008676218 is not present at all for any conditions/vertebrae combination
#### The questions is then how is 3303545110 labelled as Normal/Mild for Right Subarticular Stenosis of L1/L2 and 3008676218 labelled for all the conditions/vertebrae combination

#### Below are some of the images from the study_id/patient 100206310 

In [ ]:
dcm_image_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/100206310/1012284084/20.dcm'
left_centre_coords = (int(180.355677), int(165.0342))
right_centre_coords = (int(145.120536), int(162.285714))
radius = 10
color = (255, 0, 0)  # Red color in BGR
thickness = 2

dicom_ds = dcmread(dcm_image_path)
IMG = dicom_ds.pixel_array

IMG_normalized = cv2.normalize(IMG, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

left_IMG_with_circle = cv2.circle(IMG_normalized.copy(), left_centre_coords, radius, color, thickness)
left_IMG_with_circle = cv2.cvtColor(left_IMG_with_circle, cv2.COLOR_BGR2RGB)

right_IMG_with_circle = cv2.circle(IMG_normalized.copy(), right_centre_coords, radius, color, thickness)
right_IMG_with_circle = cv2.cvtColor(right_IMG_with_circle, cv2.COLOR_BGR2RGB)

In [ ]:
IMG.shape

In [ ]:
plt.imshow(left_IMG_with_circle)
plt.axis('off')  # Turn off axis numbers and ticks
plt.title("Left subarticular stenosis L1/L2 - Normal/Mild")
plt.show()

In [ ]:
plt.imshow(right_IMG_with_circle)
plt.axis('off')  # Turn off axis numbers and ticks
plt.title("Right subarticular stenosis L1/L2 - Normal/Mild")
plt.show()

In [ ]:
dcm_image_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/100206310/1012284084/46.dcm'
left_centre_coords = (int(168.536252), int(156.27907))
right_centre_coords = (int(146.263393), int(160.0))
radius = 10
color = (255, 0, 0)  # Red color in BGR
thickness = 2

dicom_ds = dcmread(dcm_image_path)
IMG = dicom_ds.pixel_array

IMG_normalized = cv2.normalize(IMG, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

left_IMG_with_circle = cv2.circle(IMG_normalized.copy(), left_centre_coords, radius, color, thickness)
left_IMG_with_circle = cv2.cvtColor(left_IMG_with_circle, cv2.COLOR_BGR2RGB)

right_IMG_with_circle = cv2.circle(IMG_normalized.copy(), right_centre_coords, radius, color, thickness)
right_IMG_with_circle = cv2.cvtColor(right_IMG_with_circle, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(left_IMG_with_circle)
plt.axis('off')  # Turn off axis numbers and ticks
plt.title("Left subarticular stenosis L4/L5 - Severe")
plt.show()

In [ ]:
plt.imshow(right_IMG_with_circle)
plt.axis('off')  # Turn off axis numbers and ticks
plt.title("Right subarticular stenosis L4/L5 - Moderate")
plt.show()

In [ ]:
dcm_image_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/100206310/2092806862/4.dcm'
right_centre_coords = (int(253.393075), int(278.680244))
radius = 10
color = (255, 0, 0)  # Red color in BGR
thickness = 2

dicom_ds = dcmread(dcm_image_path)
IMG = dicom_ds.pixel_array

IMG_normalized = cv2.normalize(IMG, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

right_IMG_with_circle = cv2.circle(IMG_normalized.copy(), right_centre_coords, radius, color, thickness)
right_IMG_with_circle = cv2.cvtColor(right_IMG_with_circle, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(right_IMG_with_circle)
plt.axis('off')  # Turn off axis numbers and ticks
plt.title("Right neural foraminal narrowing L4/L5 - Moderate")
plt.show()

## Aim : To select image files to create Train and Validation set
1. 25% of data will be set aside permanently as a validation set for all experiments. This set won't be used for training/testing/cross-validation
2. 75% of data will be used for all experiments
3. Images are 320 * 320 arrays
### Modelling strategy
1. Build 25 different models, one per condition/vertebrae combination

#### Pipeline to make a sample submission to test the submission data creation logic

In [ ]:
submission = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv')
submission['row_id'] = 'samples'

In [ ]:
import glob
import os

config = {}
config['root_file_path'] = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/'

studies = os.listdir(config['root_file_path'])

In [ ]:
studies

In [ ]:
rows = {}
weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in studies:
                    #print(st+'_'+s+'_'+c+'_'+v)
                    rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])
        else:
            for st in studies:
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])

In [ ]:
for row_id, feature in tqdm(rows.items()):
    feature_set_reshaped = feature.reshape(1, -1)
    predictions = np.ascontiguousarray(feature_set_reshaped/2)
    df = pd.DataFrame(predictions, columns=severity_levels)
    df.insert(loc=0, column='row_id', value=row_id)
    submission = pd.concat([submission,df]).reset_index(drop=True)
    
i = submission[(submission.row_id == 'samples')].index
submission = submission.drop(i).reset_index(drop=True)

In [ ]:
submission.to_csv('submission.csv', index=False)